练习手动搭建两层全连接神经网络
基本思路：
1.上来无从下手两眼发蒙，正常，典型的写程序少，没动过脑子，do not be scared and just trust yourself!
2.需要设置输入层、隐层、输出层的神经元的个数，设置输入，输出，权重矩阵。
3.进行n轮的epoch的迭代，关键是在每一次迭代中实现对前向和反向的计算过程。


In [11]:
!pwd

/home/control/everyday_coding_exercise/day11_22/everyday_coding_exercise


In [3]:
#第一种方法：完全用numpy实现。
import numpy as np
N,D_IN,D_H,D_OUT=64,1000,100,10
x=np.random.randn(N,D_IN)
y=np.random.randn(N,D_OUT)
#w1=np.random.rand（D_IN,D_H）
w1=np.random.randn(D_IN,D_H)
w2=np.random.randn(D_H,D_OUT)
lr=1e-6
for i in range(1):
    h=x.dot(w1)
    h_relu=np.maximum(0,h)#maximum()输入至少两个，比较对应位置，返回较大的值，np.max()输入是一个，返回最大的数
    y_pred=h_relu.dot(w2)
    
    loss=np.square(y_pred-y).sum()
    print(i,loss)
    #反向传播
    grad_y_pred=2*(y_pred-y)#从loss往第一层进行求导,该维度为（64,10），数loss对输出10维进行求导
    #print(np.shape(grad_y_pred))
    #grad_w2=grad_y_pred.dot(h_relu.T)#在grad_y_pred基础上进行下一层求导，这里涉及矩阵求导公式
    #上面的写法，不符合矩阵的规矩，虽然求导公式是这样的，注意维度！
    grad_w2=h_relu.T.dot(grad_y_pred)
    #print(np.shape(grad_w2))#(100,10)
    grad_h_relu=grad_y_pred.dot(w2.T)
    #print(np.shape(grad_h_relu))#(64,100)
    grad_h=grad_h_relu.copy()#深拷贝，对对象及其子对象都进行copy一份，对新生成的对象修改删除操作不会影响到原对象。
    #print(grad_h<0)得到一个true、false索引矩阵
    grad_h[grad_h<0]=0
    #grad_w1=grad_h.dot(x.T)#这里数学上公式是这样，但是不符合计算维度，错误同上
    grad_w1=x.T.dot(grad_h)
    #print(np.shape(grad_w1))#shape为(1000,100)
    w1-=lr*grad_w1#得到的grad_w1的维度正好和w1本身的维度一样。
    w2-=lr*grad_w2

0 30530526.097928338


In [4]:
#第二种方法：使用pytorch实现上面的一段代码
import torch
device=torch.device('cuda')
type=torch.float
N,D_IN,D_H,D_OUT=64,1000,100,10
x=torch.randn(N,D_IN,dtype=type,device=device)
y=torch.randn(N,D_OUT,dtype=type,device=device)
w1=torch.randn(D_IN,D_H,dtype=type,device=device)
w2=torch.randn(D_H,D_OUT,dtype=type,device=device)
lr=1e-6
for i in range(1):
    h=x.mm(w1)#.mm就是矩阵相乘
    h_relu=torch.clamp(h,min=0)#在pytorch官网的doc手册上进行查找，可知道每个api的含义，clamp有两种用法，按最大最小截取和按最小截取。
    y_pred=h_relu.mm(w2)
    
    loss=(y_pred-y).pow(2).sum().item()#官网手册给出的函数是torch.pow(),但是这里使用的是.pow()
    print(i,loss)
    grad_y_pred=2*(y_pred-y)
    grad_w2=h_relu.t().mm(grad_y_pred)#torch.t()表示对小于等于2维的矩阵进行转置
    grad_h_relu=grad_y_pred.mm(w2.t())
    grad_h=grad_h_relu.clone()
    grad_h[grad_h<0]=0
    grad_w1=x.t().mm(grad_h)
    
    w1-=lr*grad_w1
    w2-=lr*grad_w2

0 29053692.0


In [5]:
#第三种方法：对方法2进行改进，因为pytorch最大的优势是可以autograd，一个tensor是计算图中的一个节点，若x是一个tensor
#且x.requires_grad=True,则反向传播以后，x.grad包含着loss对于该tensor的grad信息。
import torch
device=torch.device('cuda')
type=torch.float
N,D_IN,D_H,D_OUT=64,1000,100,10
x=torch.randn(N,D_IN,dtype=type,device=device)
y=torch.randn(N,D_OUT,dtype=type,device=device)
w1=torch.randn(D_IN,D_H,dtype=type,device=device,requires_grad=True)
w2=torch.randn(D_H,D_OUT,dtype=type,device=device,requires_grad=True)
lr=1e-6
for i in range(1):
    #h=x.mm(w1)#.mm就是矩阵相乘
    #h_relu=torch.clamp(h,min=0)#在pytorch官网的doc手册上进行查找，可知道每个api的含义，clamp有两种用法，按最大最小截取和按最小截取。
    #y_pred=h_relu.mm(w2)
    
    y_pred=x.mm(w1).clamp(min=0).mm(w2)
    
    loss=(y_pred-y).pow(2).sum()#官网手册给出的函数是torch.pow(),但是这里使用的是.pow()
    print(i,loss.item())
#     grad_y_pred=2*(y_pred-y)
#     grad_w2=h_relu.t().mm(grad_y_pred)#torch.t()表示对小于等于2维的矩阵进行转置
#     grad_h_relu=grad_y_pred.mm(w2.t())
#     grad_h=grad_h_relu.clone()
#     grad_h[grad_h<0]=0
#     grad_w1=x.t().mm(grad_h)
    loss.backward()
    with torch.no_grad():#在这个模式下，所有的每一步都不计算grad，即使输入的requires_grad=True
        w1-=lr*w1.grad
        w2-=lr*w2.grad
        
        #因为在下一次迭代的时候，不需要知道w1的grad，也不需要在w1的grad上进行叠加，所以需要将其grad置0.
        w1.grad.zero_()#没有查到这个api啊？？？
        w2.grad.zero_()



0 36386432.0


In [6]:
#第四种方法：在构造网络过程中，调用torch.nn进行构造
import torch
# device=torch.device('cuda')
# type=torch.float
N,D_IN,D_H,D_OUT=64,1000,100,10
x=torch.randn(N,D_IN)
y=torch.randn(N,D_OUT)
# w1=torch.randn(D_IN,D_H,dtype=type,device=device,requires_grad=True)
# w2=torch.randn(D_H,D_OUT,dtype=type,device=device,requires_grad=True)
model=torch.nn.Sequential(
      torch.nn.Linear(D_IN,D_H),
      torch.nn.ReLU(),
      torch.nn.Linear(D_H,D_OUT)
)
loss_fn=torch.nn.MSELoss(reduction='sum')
lr=1e-4
for i in range(1):
    #h=x.mm(w1)#.mm就是矩阵相乘
    #h_relu=torch.clamp(h,min=0)#在pytorch官网的doc手册上进行查找，可知道每个api的含义，clamp有两种用法，按最大最小截取和按最小截取。
    #y_pred=h_relu.mm(w2)
    
    #y_pred=x.mm(w1).clamp(min=0).mm(w2)
    y_pred=model(x)
    
    #loss=(y_pred-y).pow(2).sum()#官网手册给出的函数是torch.pow(),但是这里使用的是.pow()
    loss=loss_fn(y_pred,y)
    print(i,loss.item())
#     grad_y_pred=2*(y_pred-y)
#     grad_w2=h_relu.t().mm(grad_y_pred)#torch.t()表示对小于等于2维的矩阵进行转置
#     grad_h_relu=grad_y_pred.mm(w2.t())
#     grad_h=grad_h_relu.clone()
#     grad_h[grad_h<0]=0
#     grad_w1=x.t().mm(grad_h)
    #进行反向传播之前，先对模型进行梯度置0，为了将这一次的梯度，不累加到下一次上。
    model.zero_grad()
    loss.backward()
    with torch.no_grad():#在这个模式下，所有的每一步都不计算grad，即使输入的requires_grad=True
#         w1-=lr*w1.grad
#         w2-=lr*w2.grad
#因为模型已经是用nn.Sequential()搭建的了，所以就不能用上述方式进行模型更新了。
        for params in model.parameters():
            #print(params)#通过打印可知道，这里的parameters就是w1.weight,b1,w2.weight,b2
            params-=lr*params.grad
        
        #因为在下一次迭代的时候，不需要知道w1的grad，也不需要在w1的grad上进行叠加，所以需要将其grad置0.
#         w1.grad.zero_()#没有查到这个api啊？？？
#         w2.grad.zero_()

0 644.18701171875


In [7]:
#第五种方法：在构造网络过程中，调用torch.nn进行构造，更新参数的时候，不手动更新，而是自动更新参数(optimizer)
import torch
# device=torch.device('cuda')
# type=torch.float
N,D_IN,D_H,D_OUT=64,1000,100,10
x=torch.randn(N,D_IN)
y=torch.randn(N,D_OUT)
# w1=torch.randn(D_IN,D_H,dtype=type,device=device,requires_grad=True)
# w2=torch.randn(D_H,D_OUT,dtype=type,device=device,requires_grad=True)
model=torch.nn.Sequential(
      torch.nn.Linear(D_IN,D_H),
      torch.nn.ReLU(),
      torch.nn.Linear(D_H,D_OUT)
)
loss_fn=torch.nn.MSELoss(reduction='sum')
lr=1e-4
optimizer=torch.optim.Adam(model.parameters(),lr)
for i in range(1):
    #h=x.mm(w1)#.mm就是矩阵相乘
    #h_relu=torch.clamp(h,min=0)#在pytorch官网的doc手册上进行查找，可知道每个api的含义，clamp有两种用法，按最大最小截取和按最小截取。
    #y_pred=h_relu.mm(w2)
    
    #y_pred=x.mm(w1).clamp(min=0).mm(w2)
    y_pred=model(x)
    
    #loss=(y_pred-y).pow(2).sum()#官网手册给出的函数是torch.pow(),但是这里使用的是.pow()
    loss=loss_fn(y_pred,y)
    print(i,loss.item())
#     grad_y_pred=2*(y_pred-y)
#     grad_w2=h_relu.t().mm(grad_y_pred)#torch.t()表示对小于等于2维的矩阵进行转置
#     grad_h_relu=grad_y_pred.mm(w2.t())
#     grad_h=grad_h_relu.clone()
#     grad_h[grad_h<0]=0
#     grad_w1=x.t().mm(grad_h)
    #进行反向传播之前，先对模型进行梯度置0，为了将这一次的梯度，不累加到下一次上。
    model.zero_grad() #貌似这里调用哪一个方法进行梯度清零都可以...
    #optimizer.zero_grad()#切记！！在反向传播之前一定将梯度置零，因为在backward的时候，grad默认是累加的！！，在每一次迭代过程中并不需要
    loss.backward()
    optimizer.step()#直接调用这个方法，就是逐步更新梯度了
#     with torch.no_grad():#在这个模式下，所有的每一步都不计算grad，即使输入的requires_grad=True
# #         w1-=lr*w1.grad
# #         w2-=lr*w2.grad
# #因为模型已经是用nn.Sequential()搭建的了，所以就不能用上述方式进行模型更新了。
#         for params in model.parameters():
#             #print(params)#通过打印可知道，这里的parameters就是w1.weight,b1,w2.weight,b2
#             params-=lr*params.grad
        
        #因为在下一次迭代的时候，不需要知道w1的grad，也不需要在w1的grad上进行叠加，所以需要将其grad置0.
#         w1.grad.zero_()#没有查到这个api啊？？？
#         w2.grad.zero_()

0 614.8849487304688


In [8]:
#第六种方法：定义一个模型（类对象），继承自nn.Module,当需要定义一个比nn.Sequential()更加复杂的模型的时候，需要这么做！
import torch
# device=torch.device('cuda')
# type=torch.float
N,D_IN,D_H,D_OUT=64,1000,100,10
x=torch.randn(N,D_IN)
y=torch.randn(N,D_OUT)
class TwoLayrNet(torch.nn.Module):
    def __init__(self,D_IN,D_H,D_OUT):#仅需要考虑一个样本输入下的情形，也就是不需要传入参数N
        super(TwoLayrNet,self).__init__()
        self.linear1=torch.nn.Linear(D_IN,D_H)
        self.linear2=torch.nn.Linear(D_H,D_OUT)
    def forward(self,x):
        h_relu=self.linear1(x).clamp(min=0)
        y_pred=self.linear2(h_relu)
        return y_pred

model=TwoLayrNet(D_IN,D_H,D_OUT)
loss_fn=torch.nn.MSELoss(reduction='sum')
lr=1e-4
optimizer=torch.optim.SGD(model.parameters(),lr)

for i in range(1):
    y_pred=model(x) 
    loss=loss_fn(y_pred,y)
    print(i,loss.item())
    optimizer.zero_grad()#切记！！在反向传播之前一定将梯度置零，因为在backward的时候，grad默认是累加的！！，在每一次迭代过程中并不需要
    loss.backward()
    optimizer.step()

0 699.8203125
